## 利用Python实现报表自动化

In [1]:
import pandas as pd
from datetime import datetime
data = pd.read_csv(r"../Data/order-14.1.csv", sep=",",engine = "python",encoding="gbk",parse_dates=["成交时间"])
data.head()
data.info()
this_month = data[(data["成交时间"]>= datetime(2018,2,1)) & (data["成交时间"]<= datetime(2018,2,28))]
last_month = data[(data["成交时间"]>= datetime(2018,1,1)) & (data["成交时间"]<= datetime(2018,1,31))]
same_month = data[(data["成交时间"]>= datetime(2017,2,1)) & (data["成交时间"]<= datetime(2017,2,28))]

def get_month_data(data):
    sale = (data["单价"]*data["销量"]).sum()
    traffic = data["订单ID"].drop_duplicates().count()
    s_t = sale/traffic
    return (sale,traffic,s_t)
sale_1,traffic_1,s_t_1 = get_month_data(this_month)
sale_2,traffic_2,s_t_2 = get_month_data(last_month)
sale_3,traffic_3,s_t_3 = get_month_data(same_month)

report = pd.DataFrame([[sale_1,sale_2,sale_3],
                       [traffic_1,traffic_2,traffic_3],
                       [s_t_1,s_t_2,s_t_3]],
                     columns = ["本月累计","上月同期","去年同期"],
                     index =["销售额","客流量","客单价"])
report

#添加同比和环比字段
report["环比"] = report["本月累计"]/report["上月同期"] -1
report["同比"] = report["本月累计"]/report["去年同期"] -1
report
report.to_csv(r"../Data/order.csv",encoding = "utf-8-sig")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3744 entries, 0 to 3743
Data columns (total 7 columns):
商品ID    3478 non-null float64
类别ID    3478 non-null float64
门店编号    3478 non-null object
单价      3478 non-null float64
销量      3478 non-null float64
成交时间    3478 non-null datetime64[ns]
订单ID    3478 non-null object
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 204.8+ KB


## 自动发送电子邮件

In [13]:
from email.mime.multipart import MIMEMultipart
import smtplib
from email import encoders
from email.header import Header
from email.mime.text import MIMEText
from email.utils import parseaddr,formataddr
from email.mime.application import MIMEApplication

#发件人邮箱
asender = "858566651@qq.com"

#收件人邮箱
areceiver = "858566651@qq.com"

#抄送人邮箱
acc = "77410764@qq.com"

#邮件主题
asubject = "这是一份测试邮件"

#发件人地址
from_addr = "858566651@qq.com"

#邮箱密码（授权码）
password = ""

#邮件设置
msg = MIMEMultipart()
msg["Subject"] = asubject
msg["to"] = areceiver
msg["Cc"] = acc
msg["from"] ="叶枫"

#邮件正文
body = "你好，这是一封测试邮件"

#添加邮件正文
msg.attach(MIMEText(body,'plain','utf-8'))

#添加附件
#注意，这里的文件路径是分割线
filename = "测试文档.xlsx"
xlsxpart = MIMEApplication(open(r'C:/Users/Administrator/Excel-Python/Data/测试文档.xlsx','rb').read())
xlsxpart.add_header('Content-Disposition', 'attachment',filename=filename) 
msg.attach(xlsxpart)


#设置邮箱服务器地址及端口
smtp_server= "smtp.qq.com"
server = smtplib.SMTP(smtp_server,25)
server.set_debuglevel(1)

#登录邮箱
server.login(from_addr,password)
server.sendmail(from_addr,areceiver.split(",")+acc.split(","),msg.as_string())

#断开服务器连接
server.quit()



# file_mail = MIMEApplication(open(fileurl,'rb').read())  
# file_mail.add_header('Content-Disposition', 'attachment',filename=filename.decode('utf-8').encode('gb2312'))  
# msg.attach(file_mail)


# part = MIMEBase('application', 'octet-stream') #'octet-stream': binary data 
# part.set_payload(open(file, 'rb').read())
# encoders.encode_base64(part)
# part.add_header('Content-Disposition', 'attachment; filename="%s"' % os.path.basename(file)) 
# msg.attach(part)

send: 'ehlo [172.30.48.1]\r\n'
reply: b'250-smtp.qq.com\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-SIZE 73400320\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-AUTH LOGIN PLAIN\r\n'
reply: b'250-AUTH=LOGIN\r\n'
reply: b'250-MAILCOMPRESS\r\n'
reply: b'250 8BITMIME\r\n'
reply: retcode (250); Msg: b'smtp.qq.com\nPIPELINING\nSIZE 73400320\nSTARTTLS\nAUTH LOGIN PLAIN\nAUTH=LOGIN\nMAILCOMPRESS\n8BITMIME'
send: 'AUTH PLAIN ADg1ODU2NjY1MUBxcS5jb20AcHhjbndyYXVocWFlYmNqaA==\r\n'
reply: b'235 Authentication successful\r\n'
reply: retcode (235); Msg: b'Authentication successful'
send: 'mail FROM:<858566651@qq.com> size=8627\r\n'
reply: b'250 Ok\r\n'
reply: retcode (250); Msg: b'Ok'
send: 'rcpt TO:<858566651@qq.com>\r\n'
reply: b'250 Ok\r\n'
reply: retcode (250); Msg: b'Ok'
send: 'rcpt TO:<77410764@qq.com>\r\n'
reply: b'250 Ok\r\n'
reply: retcode (250); Msg: b'Ok'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: b'End data with <CR><LF>.<CR><LF>'
data:

reply: b'250 Ok: queued as \r\n'
reply: retcode (250); Msg: b'Ok: queued as'
data: (250, b'Ok: queued as')
send: 'quit\r\n'
reply: b'221 Bye\r\n'
reply: retcode (221); Msg: b'Bye'


(221, b'Bye')